# Connect to MLflow in Databricks

> Note that Mlflow reccomends using `ChatAgent` instead of `ChatModel`.

In [ ]:
import mlflow
import mlflow.tracking._model_registry.utils
from dotenv import load_dotenv

load_dotenv()

# The login and mlflow model registery are set this way because of local development. If you are running this in Databricks, you can remove the login and use the regualr mlflow registry.

mlflow.login()

mlflow.tracking._model_registry.utils._get_registry_uri_from_spark_session = (
    lambda: "databricks-uc"
)

mlflow.set_experiment("/ChatModel")

# Register the model as code

In [ ]:
with mlflow.start_run():
    model_info = mlflow.pyfunc.log_model(
        artifact_path="model",
        streamable=True,
        pip_requirements="../../requirements.txt",
        python_model="ChatModel.py",
        input_example={
            "messages": [{"role": "user", "content": "Hello, how are you?"}]
        },
    )

my_model = mlflow.pyfunc.load_model(model_info.model_uri)

# Test the registered model

In [ ]:
result = my_model.predict(
    data={
        "messages": [{"role": "user", "content": "What color is the sky?"}],
    }
)

result

In [ ]:
stream_generator = my_model.predict_stream(
    data={
        "messages": [{"role": "user", "content": "What color is the sky?"}],
    }
)

for response in stream_generator:
    print(response)